In [ ]:
!pip install -U transformers
!pip install rouge-score nltk

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=ba1b6b53ac4090a46499a13d5ad2df45b65cca28917834f0ac7294869dacfe59
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [ ]:
import re
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, TFT5ForConditionalGeneration
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu

## Dataset

The Warren Buffett Letters Q&A Enhanced Dataset (1998-2024) is a curated collection of question-answer pairs derived from Warren Buffett's annual shareholder letters to Berkshire Hathaway. The dataset spans 26 years of Buffett's investment wisdom and contains structured Q&A pairs extracted from real letter excerpts for conversational learning applications.

**Column Descriptions:**
- **question**: Financial queries covering investment philosophy, business analysis, market observations, and strategic decisions based on Buffett's letter content
- **answer**: Expert-level responses reflecting Buffett's investment principles, analytical approach, and decision-making framework
- **reasoning**: Detailed explanations of the underlying logic and investment principles behind each answer, providing context from Buffett's methodology

The dataset was chosen for its authentic representation of Warren Buffett's investment philosophy and its structured format that enables effective fine-tuning of sequence-to-sequence models for financial advisory applications in the value investing domain.

In [ ]:
df = pd.read_parquet("hf://datasets/eagle0504/warren-buffett-letters-qna-r1-enhanced-1998-2024/data/train-00000-of-00001.parquet")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
df.head()

,question,answer,reasoning
0,"**""How does Warren Buffett's principle of 'pra...",A good answer would be: \n\n*Warren Buffett e...,The reasoning is as follows: \n\n1. **Context...
1,Here are a few strong questions worth asking b...,A good answer would be: \n\nWarren Buffett ac...,"Warren Buffett emphasizes transparency, accoun..."
2,"**""How does Warren Buffett's principle of 'pra...",Here’s a concise answer derived from the parag...,The reasoning is as follows: \n\n1. **Context...
3,"**""How does Warren Buffett's principle of 'pra...",A good answer would highlight Warren Buffett's...,The reasoning is as follows: \n\n1. **Context...
4,"**""How does Warren Buffett's principle of 'pra...",A good answer would highlight Warren Buffett's...,The reasoning is as follows: \n\n1. **Context...


## Model

FLAN-T5 is an enhanced version of Google's T5 (Text-to-Text Transfer Transformer) architecture. It extends the original T5 by fine-tuning on over 1,000 additional tasks across multiple languages, creating a more robust foundation model for natural language processing applications.

**Key advantages of FLAN-T5:**
- **Multilingual capabilities**: Trained on diverse language datasets for broader applicability  
- **Sequence-to-sequence excellence**: Proven effectiveness for conditional text generation tasks
- **Large-scale pre-training**: Extensive training corpus ensures strong foundational knowledge

FLAN-T5 was selected as the base model due to its demonstrated performance on sequence-to-sequence tasks and its ability to follow instructions effectively, making it well-suited for question-answering applications.

**Base Model**: [google/flan-t5-base](https://huggingface.co/google/flan-t5-base)

In [ ]:
def load_tokenizer(model_name="google/flan-t5-base"):
    """
    Loads tokenizer for T5 model from HuggingFace Hub.

    Args:
        model_name: HuggingFace model identifier (default: "google/flan-t5-base")

    Returns:
        AutoTokenizer: Pre-trained tokenizer matching the specified model
    """
    # Load compatible tokenizer for text preprocessing
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return tokenizer


In [ ]:
def load_model(model_name="google/flan-t5-base"):
    """
    Loads pre-trained T5 model from HuggingFace Hub.

    Args:
        model_name: HuggingFace model identifier (default: "google/flan-t5-base")

    Returns:
        TFT5ForConditionalGeneration: Loaded TensorFlow T5 model for conditional generation
    """
    # Load pre-trained T5 model for sequence-to-sequence tasks
    model = TFT5ForConditionalGeneration.from_pretrained(model_name)
    return model

## Preprocessing

The dataset undergoes comprehensive preprocessing to ensure high-quality training data for the sequence-to-sequence model. The preprocessing pipeline consists of dataset combination, text cleaning, and quality filtering.

**Dataset Enhancement:**
The original financial Q&A data is combined with 75+ out-of-domain questions paired with standardized rejection responses, teaching the model to recognize and politely decline non-financial queries.

**Text Cleaning:**
Removes markdown formatting, escaped characters, and annotations while preserving content. Normalizes whitespace and extracts relevant questions from formatted text to create clean, uniform input.

**Quality Filtering:**
Eliminates entries with missing data and enforces minimum length requirements for financial content while preserving all rejection examples for robust domain boundary learning.

This preprocessing approach creates clean input that enables effective model training while teaching proper domain boundaries and preserving Buffett's investment insights.

In [ ]:
def preprocess_for_chatbot(text):
    """
    Preprocesses text for chatbot input by cleaning formatting and extracting questions.

    Args:
        text: Raw text input (may contain markdown, escape characters, annotations)

    Returns:
        str: Cleaned text ready for chatbot processing
    """
    # Handle missing or null values
    if pd.isna(text):
        return ""

    text = str(text)

    # Replace escaped characters for readability
    text = text.replace("\\n", " ").replace("\\t", " ").replace("\\'", "'").replace('\\"', '"')

    # Extract first question from bold markdown format
    questions = re.findall(r'\*\*"([^"]*?)"\*\*', text)
    if questions:
        return questions[0].strip()

    # Remove markdown formatting
    text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)  # **text** → text
    text = re.sub(r'\*(.*?)\*', r'\1', text)      # *text* → text

    # Remove annotations in parentheses
    text = re.sub(r'\*\([^)]*\)\*', '', text)

    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove surrounding quotes if they wrap the entire text
    if text.startswith('"') and text.endswith('"') and text.count('"') == 2:
        text = text[1:-1].strip()

    return text

In [ ]:
def preprocess_chatbot_dataset(df):
    """
    Preprocesses the entire chatbot dataset by cleaning text and filtering entries.

    Args:
        df: DataFrame containing question-answer pairs with optional reasoning

    Returns:
        DataFrame: Cleaned dataset ready for model training
    """
    # Apply text cleaning to all relevant columns
    for col in ['question', 'reasoning', 'answer']:
        if col in df.columns:
            df[col] = df[col].apply(preprocess_for_chatbot)

    # Remove entries with missing critical data
    df = df.dropna(subset=['question', 'answer'])

    # Filter out entries that are too short to be meaningful
    df = df[df['question'].str.len() > 5]
    df = df[df['answer'].str.len() > 10]

    print(f"Dataset ready: {len(df)} samples")

    # Display sample data for verification
    if len(df) > 0:
        print(f"\nSample preserved question:")
        print(f"'{df['question'].iloc[0]}'")
        print(f"\nSample preserved answer:")
        print(f"'{df['answer'].iloc[0]}'")
        print(f"\nSample preserved reasoning:")
        print(f"'{df['reasoning'].iloc[0]}'")

    return df

In [ ]:
from sklearn.model_selection import train_test_split

def split_train_val(df, val_size=0.1):
    """
    Splits dataset into training and validation sets.

    Args:
        df: Input DataFrame to split
        val_size: Fraction for validation set (default: 0.2)

    Returns:
        tuple: (train_df, val_df) - Training and validation DataFrames
    """
    # Split with fixed random seed
    train_df, val_df = train_test_split(df, test_size=val_size, random_state=42)

    return train_df, val_df

In [ ]:
def tokenize_data(df, tokenizer, batch_size=16):
    """
    Tokenizes question-answer pairs and returns a batched tf.data.Dataset.

    Args:
        df: DataFrame with 'question' and 'answer' columns
        tokenizer: Pre-trained tokenizer (e.g., from transformers library)
        batch_size: Number of samples per batch (default: 4)

    Returns:
        tf.data.Dataset: Batched dataset with input_ids, attention_mask, and labels
    """
    inputs = []
    labels = []
    attention_masks = []

    for index, row in df.iterrows():
        # Add context prompt during training (same as inference)
        input_text = f"Answer this financial question based on Warren Buffett's principles: {row['question']}"

        question_tokens = tokenizer(
            input_text,  # Use formatted input instead of raw question
            max_length=256,
            padding='max_length',
            truncation=True,
            return_tensors='tf'
        )
        inputs.append(question_tokens['input_ids'][0])
        attention_masks.append(question_tokens['attention_mask'][0])

        # Tokenize answer
        answer_tokens = tokenizer(
            row['answer'],
            max_length=256,
            padding='max_length',
            truncation=True,
            return_tensors='tf'
        )
        labels.append(answer_tokens['input_ids'][0])

    # Convert to tensors
    inputs = tf.stack(inputs)
    attention_masks = tf.stack(attention_masks)
    labels = tf.stack(labels)

    # Create and return batched dataset
    return tf.data.Dataset.from_tensor_slices({
        'input_ids': inputs,
        'attention_mask': attention_masks,
        'labels': labels
    }).batch(batch_size)

In [ ]:
# Load the out-of-domain data
out_of_domain_df = pd.read_csv('out_of_domain_training_data.csv')

# Combine both datasets
enhanced_df = pd.concat([df, out_of_domain_df], ignore_index=True)

# Shuffle the combined dataset
enhanced_df = enhanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
df = preprocess_chatbot_dataset(enhanced_df)

Dataset ready: 10741 samples

Sample preserved question:
'What's a good recipe for pizza?'

Sample preserved answer:
'I'm designed to answer questions about Warren Buffett's investment principles and philosophy. Could you please ask a question related to investing, business analysis, or financial strategy?'

Sample preserved reasoning:
'Out-of-domain query requiring polite redirection to financial topics'


In [ ]:
train_df, val_df = split_train_val(df)

print(f"Train samples: {len(train_df)}")
print(f"Val samples: {len(val_df)}")

Train samples: 9666
Val samples: 1075


In [26]:
def train_model(train_dict, val_dict, learning_rate=5.8e-5, callbacks=[], epochs=30):
    """
    Fine-tunes FLAN-T5 model on question-answer pairs.

    Args:
        train_dict: Training dataset (tf.data.Dataset with input_ids, attention_mask, labels)
        val_dict: Validation dataset (tf.data.Dataset with same structure)
        learning_rate: Adam optimizer learning rate (default: 5e-5)
        callbacks: List of Keras callbacks for training monitoring (default: [])
        epochs: Number of training epochs (default: 35)

    Returns:
        TFT5ForConditionalGeneration: Fine-tuned model ready for inference
    """
    # Load pre-trained FLAN-T5 base model
    model = load_model()

    # Configure Adam optimizer with specified learning rate
    optimizer = tf.keras.optimizers.AdamW(
        learning_rate=learning_rate,
        weight_decay=0.01
        )
    model.compile(optimizer=optimizer)

    # Fine-tune model with callbacks for training monitoring
    model.fit(
        train_dict,
        validation_data=val_dict,
        epochs=epochs,
        callbacks=callbacks
    )

    return model

In [27]:
def get_simple_callbacks():
    """
    Creates standard Keras callbacks for training optimization and monitoring.

    Returns:
        list: Configured callbacks for early stopping, learning rate reduction, and model checkpointing
    """
    # Training hyperparameters - easily adjustable
    PATIENCE = 3             # Epochs to wait before early stopping
    LR_REDUCTION_FACTOR = 0.3 # Factor to reduce learning rate by
    LR_PATIENCE = 1           # Epochs to wait before reducing learning rate
    MIN_LR = 1e-8            # Minimum learning rate threshold
    MIN_DELTA = 0.001         # Minimum change to qualify as improvement
    RESTORE_BEST = True      # Restore best weights on early stop

    callbacks = [
        # Prevent overfitting by stopping training when validation loss plateaus
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=PATIENCE,
            restore_best_weights=RESTORE_BEST,
            min_delta=MIN_DELTA,
            verbose=1
        ),

        # Adaptive learning rate reduction when training stagnates
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=LR_REDUCTION_FACTOR,
            patience=LR_PATIENCE,
            min_lr=MIN_LR,
            min_delta=MIN_DELTA,
            cooldown=1,              # Wait 1 epoch after reduction
            verbose=1
        ),

        # tf.keras.callbacks.LearningRateScheduler(
        #     lambda epoch: 3e-5 * (0.9 ** epoch)
        # ),

        # Save best performing model during training
        tf.keras.callbacks.ModelCheckpoint(
            filepath='./best_model.keras',
            monitor='val_loss',
            save_best_only=True,
            verbose=1
        )
    ]

    return callbacks

In [28]:
tokenizer = load_tokenizer()

train_dict = tokenize_data(train_df, tokenizer)
val_dict = tokenize_data(val_df, tokenizer)

In [29]:
callbacks = get_simple_callbacks()

model = train_model(train_dict, val_dict, callbacks=callbacks)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Epoch 1/30
605/605 [==============================] - ETA: 0s - loss: 1.8589
Epoch 1: val_loss improved from inf to 0.80000, saving model to ./best_model.keras
605/605 [==============================] - 257s 309ms/step - loss: 1.8589 - val_loss: 0.8000 - lr: 5.8000e-05
Epoch 2/30
605/605 [==============================] - ETA: 0s - loss: 0.8279
Epoch 2: val_loss improved from 0.80000 to 0.72377, saving model to ./best_model.keras
605/605 [==============================] - 167s 276ms/step - loss: 0.8279 - val_loss: 0.7238 - lr: 5.8000e-05
Epoch 3/30
605/605 [==============================] - ETA: 0s - loss: 0.7576
Epoch 3: val_loss improved from 0.72377 to 0.66987, saving model to ./best_model.keras
605/605 [==============================] - 167s 276ms/step - loss: 0.7576 - val_loss: 0.6699 - lr: 5.8000e-05
Epoch 4/30
605/605 [==============================] - ETA: 0s - loss: 0.7047
Epoch 4: val_loss improved from 0.66987 to 0.62526, saving model to ./best_model.keras
605/605 [=========

In [30]:
question = "How often do you review your portfolio"

In [31]:
def predict_answer(question, model, tokenizer):
    """
    Generates answers to financial questions using fine-tuned FLAN-T5 model.

    Args:
        question: Input question text
        model: Fine-tuned TFT5ForConditionalGeneration model
        tokenizer: Corresponding T5 tokenizer

    Returns:
        str: Generated answer following Warren Buffett's investment principles
    """
    # Format input with same context prompt used during training
    input_text = f"Answer this financial question based on Warren Buffett's principles: {question}"

    # Tokenize input with padding and truncation for consistent format
    input_tokens = tokenizer(
        input_text,
        return_tensors="tf",
        max_length=256,
        padding='max_length',
        truncation=True
    )

    # Generate response using beam search for higher quality output
    generated_tokens = model.generate(
        input_tokens["input_ids"],
        attention_mask=input_tokens["attention_mask"],
        max_length=256,
        num_beams=4,
        early_stopping=True
    )

    # Decode generated tokens to readable text
    predicted_answer = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

    return predicted_answer

# Example usage
predicted_answer = predict_answer(question, model, tokenizer)
print(f"\nQuestion: {question}")
print(f"Predicted Answer: {predicted_answer}")


Question: How often do you review your portfolio
Predicted Answer: How does Berkshire ensure that the questions selected by analysts and journalists truly reflect the most pressing concerns of shareholders, rather than just the interests of the financial media?


In [32]:
def calculate_bleu(reference, candidate):
    """
    Calculates BLEU score to measure similarity between reference and generated text.

    Args:
        reference: Ground truth answer text
        candidate: Model-generated answer text

    Returns:
        float: BLEU score between 0 and 1 (higher = better similarity)
    """
    # Tokenize texts by splitting on whitespace
    reference_tokens = reference.split()
    candidate_tokens = candidate.split()

    # Calculate sentence-level BLEU score
    return sentence_bleu([reference_tokens], candidate_tokens)

In [33]:
def calculate_rouge(reference, candidate):
    """
    Calculates ROUGE scores to evaluate text generation quality.

    Args:
        reference: Ground truth answer text
        candidate: Model-generated answer text

    Returns:
        dict: ROUGE scores with keys 'rouge1', 'rouge2', 'rougeL' (0-1 scale, higher = better)
    """
    # Initialize ROUGE scorer with stemming for better matching
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    # Calculate all ROUGE variants
    scores = scorer.score(reference, candidate)

    # Return F1 scores for each ROUGE metric
    return {
        'rouge1': scores['rouge1'].fmeasure,   # Unigram overlap
        'rouge2': scores['rouge2'].fmeasure,   # Bigram overlap
        'rougeL': scores['rougeL'].fmeasure    # Longest common subsequence
    }

In [34]:
import math

def calculate_perplexity(model, tokenizer, text, max_length=256):
    """
    Calculates perplexity to measure how well the model predicts the given text.

    Args:
        model: Fine-tuned language model
        tokenizer: Corresponding tokenizer
        text: Input text to evaluate
        max_length: Maximum sequence length for tokenization (default: 256)

    Returns:
        float: Perplexity score (lower = better, inf if calculation fails)
    """
    try:
        # Tokenize input text with truncation for consistent length
        inputs = tokenizer(text, return_tensors='tf', max_length=max_length, truncation=True)
        input_ids = inputs['input_ids']

        # Calculate loss using input as both input and target (language modeling)
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss

        # Convert cross-entropy loss to perplexity
        perplexity = math.exp(loss.numpy())
        return perplexity

    except:
        # Return infinity if calculation fails (e.g., empty text, model errors)
        return float('inf')

In [35]:
def evaluate_model(model, tokenizer, test_df, sample_number=10):
    """
    Evaluates model performance using multiple metrics on test data.

    Args:
        model: Fine-tuned model to evaluate
        tokenizer: Corresponding tokenizer
        test_df: Test dataset with 'question' and 'answer' columns
        sample_number: Number of samples to evaluate (default: 10)

    Returns:
        dict: Average scores for BLEU, ROUGE variants, and perplexity
    """
    bleu_scores = []
    rouge_scores = []
    perplexity_scores = []

    # Sample subset of test data for evaluation with fixed random seed
    test_df = test_df.sample(n=sample_number, random_state=42)

    for _, row in test_df.iterrows():
        # Use your predict_answer function for consistency
        prediction = predict_answer(row['question'], model, tokenizer)

        # Calculate evaluation metrics
        bleu = calculate_bleu(row['answer'], prediction)
        rouge = calculate_rouge(row['answer'], prediction)
        perplexity = calculate_perplexity(model, tokenizer, row['answer'])

        # Store scores for averaging
        bleu_scores.append(bleu)
        rouge_scores.append(rouge)
        perplexity_scores.append(perplexity)

    # Filter out infinite perplexity values
    valid_perplexities = [p for p in perplexity_scores if p != float('inf')]

    # Calculate average scores across all samples
    avg_bleu = np.mean(bleu_scores)
    avg_rouge1 = np.mean([r['rouge1'] for r in rouge_scores])
    avg_rouge2 = np.mean([r['rouge2'] for r in rouge_scores])
    avg_rougeL = np.mean([r['rougeL'] for r in rouge_scores])
    avg_perplexity = np.mean(valid_perplexities) if valid_perplexities else float('inf')

    # Display evaluation results
    print(f"BLEU Score: {avg_bleu:.4f}")
    print(f"ROUGE-1: {avg_rouge1:.4f}")
    print(f"ROUGE-2: {avg_rouge2:.4f}")
    print(f"ROUGE-L: {avg_rougeL:.4f}")
    print(f"Perplexity: {avg_perplexity:.2f}")

    return {
        'bleu': avg_bleu,
        'rouge1': avg_rouge1,
        'rouge2': avg_rouge2,
        'rougeL': avg_rougeL,
        'perplexity': avg_perplexity
    }

In [36]:
metrics = evaluate_model(model, tokenizer, val_df)

/tmp/ipython-input-34-210207630.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  perplexity = math.exp(loss.numpy())


BLEU Score: 0.1148
ROUGE-1: 0.3763
ROUGE-2: 0.1862
ROUGE-L: 0.2976
Perplexity: 1.27


In [37]:
from datetime import datetime

def save_model(model, tokenizer, name="my_model"):
    """
    Saves fine-tuned model and tokenizer with timestamped directory name.

    Args:
        model: Fine-tuned TFT5ForConditionalGeneration model
        tokenizer: Corresponding T5 tokenizer
        name: Base name for the saved model (default: "my_model")

    Returns:
        None: Creates timestamped directory in ./models/ with saved model files
    """
    # Generate timestamp for unique model versioning
    timestamp = datetime.now().strftime("%m%d_%H%M")
    full_name = f"{name}_{timestamp}"

    # Save model and tokenizer to timestamped directory
    model.save_pretrained(f"./models/{full_name}")
    tokenizer.save_pretrained(f"./models/{full_name}")

    print(f"Saved: {full_name}")

# Example usage
save_model(model, tokenizer, name="finance_chatbot")

Saved: finance_chatbot_0619_1229
